# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [5]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter,EnhancedTextFileLoader
from aimakerspace.vectordatabase import VectorDatabase, SimpleEnhancedVectorDatabase
import asyncio

In [6]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [7]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [8]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [9]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [10]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [11]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key



### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1.We can modify the dimension of the embedding ( without the embedding losing its concept-representing properties) by passing `dimensions` parameter (only supported in text-embedding-3 and later models). for example:

client.embeddings.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter...",
  encoding_format="float"
  dimensions=1024 
). 

Also we can change the dimension after generating the embedding like:

def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)


response = client.embeddings.create(
    model="text-embedding-3-small", input="Testing 123", encoding_format="float"
)

cut_dim = response.data[0].embedding[:256]
norm_dim = normalize_l2(cut_dim)

2.The technique that OpenAI use to achive this is:Matryoshka Representation Learning, which encodes information at
different granularities and allows a single embedding to adapt to the computational constraints of downstream tasks.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [24]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
Each request to generate an embedding is executed one after another. Your code waits for the current request to finish before moving to the next one.When collecting embeddings, synchronous (sync) execution processes requests one by one, causing delays, while asynchronous (async) execution handles multiple requests concurrently, significantly improving speed and throughput, especially for I/O-bound tasks like fetching data or communicating with embedding models. Async is the optimal choice for embedding large datasets because it utilizes idle time and keeps tasks moving without blocking, whereas sync processing would lead to long wait times.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [25]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('tings of the learned Matryoshka Representations .\nAdaptive RetrievalShortlistingRe-ranking\nAdaptive ClassificationTrainingInference\n<latexit sha1_base64="eh9hk+peBkdsPY6v+r4rONmxYLY=">AAAB7nicbVBNSwMxEJ2tX7V+VT16CRbBU9kVoR6LXjxWsB/QLiWbZtvQbBKSrFCW/ggvHhTx6u/x5r8x2+5BWx8MPN6bYWZepDgz1ve/vdLG5tb2Tnm3srd/cHhUPT7pGJlqQttEcql7ETaUM0HblllOe0pTnEScdqPpXe53n6g2TIpHO1M0TPBYsJgRbJ3UHUjFU1MZVmt+3V8ArZOgIDUo0BpWvwYjSdKECks4NqYf+MqGGdaWEU7nlUFqqMJkise076jACTVhtjh3ji6cMkKx1K6ERQv190SGE2NmSeQ6E2wnZtXLxf+8fmrjmzBjQqWWCrJcFKccWYny39GIaUosnzmCiWbuVkQmWGNiXUJ5CMHqy+ukc1UP/HrwcF1r3hZxlOEMzuESAmhAE+6hBW0gMIVneIU3T3kv3rv3sWwtecXMKfyB9/kDBrKPWQ==</latexit>\n<latexit sha1_base64="szvlzTDYQEH5M/GSBtp0D9XaTiU=">AAAB/HicbVDLSsNAFL2pr1pf0S7dDBahbkoiguKq6MaFiwr2AW0Ik8m0HTp5MDMRYqi/4saFIm79EHf+jZM2C209MHA4517umePFnEllWd9GaWV1bX2jvFnZ2t7Z3TP3DzoySgShbRLxSPQ8LClnIW0rpjjtxYLiwOO0602uc7/7QIVkUXiv0pg6AR6FbMgIVlpyzeogwGpMMEe39Uc3sy/96Ylr1qyGNQNaJnZBalCg5ZpfAz8iSUBDRTiWsm9bsXIyLBQjnE4rg0TSGJMJHtG+piEOqHSyWfgpOtaKj4

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
There are some ways to make more reproducible outputs:
-Prompt engineering:writing effective instructions for a model. Some tools and techniques to construnct prompts:
Choosing models like gpt-5, gpt-5-mini,gpt--nano, reasoning models like o3 and GPT-5, respond better to different prompts. Giving instructions parameter along with message roles gives the model high level instructions on how it should behave while generating a response, including tone, goals, and examples of correct responses. for example:
response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)
Three types of role describes how models give different levels of priority to messages: develope, user, assistant.
The best way is to create reusabe prompts rather than hardcoded.

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [14]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [15]:
print(response)

Hello! The "best" way to write a loop in Python depends on what you're trying to accomplish. Generally, Python offers a few clean and readable ways to write loops:

1. **for loop** – often used to iterate over a sequence (like a list, tuple, string, or range).
   ```python
   for item in iterable:
       # do something with item
   ```

2. **while loop** – useful when you want to repeat something as long as a condition holds.
   ```python
   while condition:
       # do something
   ```

For example, if you want to loop over numbers from 0 to 4:
```python
for i in range(5):
    print(i)
```

Or, a while loop that counts down:
```python
count = 5
while count > 0:
    print(count)
    count -= 1
```

In general, using a `for` loop with iterables or `range` is preferred for readability and simplicity. If you have more specific needs or a particular scenario in mind, I’d be happy to help tailor the loop to that!


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [16]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [17]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [18]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder hires a weak executive to run the function or specialty that originally brought them to their leadership role. This happens because the CEO has difficulty letting go of that function and wants to remain "the man" or the key figure in that area. As a result, they end up hiring someone weak on purpose in that executive role. The problem is named after Michael Eisner, the CEO of Disney who had been a brilliant TV network executive; when he acquired ABC, the network fell to fourth place, and his response was that if he had more time, he could turn it around himself. However, he did not have the extra time, illustrating the issue of a CEO trying to retain control over their former specialty instead of empowering a strong executive ("Source 1").

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
The prompting strategies to make the LLM have a more thoughtful, detailed response is to adjust the reasoning effort parameter:reasoning={"effort": "low" | "medium" | "high"} – Controls how much computational effort the model puts into reasoning. low favors speed and economical token usage, and high favors more complete reasoning. The default value is medium, which is a balance between speed and reasoning accuracy. This is called Reasoning Control.

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [ ]:
from aimakerspace.text_utils import EnhancedTextFileLoader

# Test the PDF loader with MRL.pdf
print("🔍 Testing Enhanced PDF Loader:")
print("=" * 40)

try:
    # Load the PDF using the enhanced loader
    pdf_loader = EnhancedTextFileLoader("data/MRL.pdf")
    pdf_documents = pdf_loader.load_documents()
    print(f"📊 Number of documents: {len(pdf_documents)}")
    
except Exception as e:
    
    print("📝 Using original text documents as fallback")

print(f"\n🎯 Ready to use {len(pdf_documents)} document(s) with enhanced RAG!")

🔍 Testing Enhanced PDF Loader:
📊 Number of documents: 1

🎯 Ready to use 1 document(s) with enhanced RAG!


In [ ]:

class SimpleEnhancedRAGPipeline(RetrievalAugmentedQAPipeline):
    """Simple enhancement to existing RAG pipeline - just adds distance metric choice"""
    
    def run_pipeline_with_metric(self, user_query: str, k: int = 4, distance_measure: str = 'cosine'):
        """Run pipeline with chosen distance metric"""
        
        # Use the enhanced search if available
        if hasattr(self.vector_db_retriever, 'search_with_metric'):
            context_list = self.vector_db_retriever.search_with_metric(user_query, k=k, distance_measure=distance_measure)
        else:
            # Fallback to regular search
            context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        # Use existing pipeline logic
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Enhanced system prompt with metric info
        system_params = {
            "response_style": self.response_style,
            "response_length": "detailed"
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        # Add metric info to user prompt
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores ({distance_measure}): {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)
        
        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]),
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores,
            "distance_measure": distance_measure
        }



In [39]:
try:
    text_splitter = CharacterTextSplitter()
    split_documents = text_splitter.split_texts(pdf_documents)
    
    # Create the simple enhanced database
    simple_enhanced_db = SimpleEnhancedVectorDatabase()
    simple_enhanced_db = asyncio.run(simple_enhanced_db.abuild_from_list(split_documents[:20]))  # First 20 for demo
    
    print(f"✅ Enhanced database built with {len(split_documents[:20])} chunks")
    print(f"📏 Available metrics: {list(simple_enhanced_db.distance_metrics.keys())}")
    
except Exception as e:
    print(f"❌ Using fallback: {e}")
    simple_enhanced_db = vector_db  

# 3. Test different distance metrics
print("\n3. 📏 Testing Distance Metrics...")
test_query = "What is the Michael Eisner Memorial Weak Executive Problem?"

for metric in ['cosine', 'euclidean', 'manhattan']:
    try:
        if hasattr(simple_enhanced_db, 'search_with_metric'):
            results = simple_enhanced_db.search_with_metric(test_query, k=3, distance_measure=metric)
            print(f"✅ {metric.upper()}: {len(results)} results, top score: {results[0][1]:.3f}")
        else:
            print(f"⚠️ {metric}: using fallback search")
    except Exception as e:
        print(f"❌ {metric}: {e}")

# 4. Test enhanced RAG pipeline
print("\n4. 🤖 Testing Enhanced RAG...")
try:
    # Create simple enhanced RAG using components
    simple_enhanced_rag = SimpleEnhancedRAGPipeline(
        llm=chat_openai,  # Your existing LLM
        vector_db_retriever=simple_enhanced_db,
        response_style="detailed",
        include_scores=True
    )
    
    # Test with different metrics
    for metric in ['cosine', 'euclidean']:
        print(f"\n🔍 Testing {metric.upper()}:")
        
        result = simple_enhanced_rag.run_pipeline_with_metric(
            test_query, 
            k=3, 
            distance_measure=metric
        )
        
        print(f"📝 Response: {result['response'][:200]}...")
        print(f"📊 Used {result['distance_measure']} metric")
        print(f"📈 Scores: {result['similarity_scores'][:2]}")
        
except Exception as e:
    print(f"❌ Enhanced RAG failed: {e}")


✅ Enhanced database built with 20 chunks
📏 Available metrics: ['cosine', 'euclidean', 'manhattan']

3. 📏 Testing Distance Metrics...
✅ COSINE: 3 results, top score: 0.259
✅ EUCLIDEAN: 3 results, top score: 0.451
✅ MANHATTAN: 3 results, top score: 0.026

4. 🤖 Testing Enhanced RAG...

🔍 Testing COSINE:
📝 Response: I don't know. The provided context does not contain any information about the Michael Eisner Memorial Weak Executive Problem....
📊 Used cosine metric
📈 Scores: ['Source 1: 0.259', 'Source 2: 0.241']

🔍 Testing EUCLIDEAN:
📝 Response: I don't know....
📊 Used euclidean metric
📈 Scores: ['Source 1: 0.451', 'Source 2: 0.448']


In [40]:
print("🧪 Testing with PDF-appropriate queries:")

if pdf_documents:
    sample_text = pdf_documents[0][:500]
    print(f"📄 PDF sample content:\n{sample_text}\n")
    
    pdf_query = "What does this document discuss?"  
    
    print(f"🔍 Testing with PDF-appropriate query...")
    for metric in ['cosine', 'euclidean', 'manhattan']:
        try:
            results = enhanced_db.search_with_metric(pdf_query, k=3, distance_measure=metric)
            print(f"✅ {metric.upper()}: top score: {results[0][1]:.3f}")
        except Exception as e:
            print(f"❌ {metric}: {e}")

🧪 Testing with PDF-appropriate queries:
📄 PDF sample content:

--- Page 1 ---
Matryoshka Representation Learning
Aditya Kusupati∗†⋄, Gantavya Bhatt∗†, Aniket Rege∗†,
Matthew Wallingford†, Aditya Sinha⋄, Vivek Ramanujan†, William Howard-Snyder†,
Kaifeng Chen⋄, Sham Kakade‡, Prateek Jain⋄and Ali Farhadi†
†University of Washington,⋄Google Research,‡Harvard University
{kusupati,ali}@cs.washington.edu ,prajain@google.com
Abstract
Learned representations are a central component in modern ML systems, serv-
ing a multitude of downstream tasks. When training such r

🔍 Testing with PDF-appropriate query...
✅ COSINE: top score: 0.265
✅ EUCLIDEAN: top score: 0.452
✅ MANHATTAN: top score: 0.026


In [41]:
# Complete Working Test - All in One Cell
from aimakerspace.vectordatabase import SimpleEnhancedVectorDatabase
from aimakerspace.text_utils import CharacterTextSplitter
import asyncio
import importlib
import aimakerspace.vectordatabase

importlib.reload(aimakerspace.vectordatabase)
from aimakerspace.vectordatabase import SimpleEnhancedVectorDatabase

print("🚀 Complete Enhanced RAG Test")
print("=" * 40)

# 1. Check if we have PDF documents
try:
    if 'pdf_documents' in globals() and pdf_documents:
        print(f"✅ Using {len(pdf_documents)} PDF document(s)")
        source_docs = pdf_documents
    else:
        print("⚠️ PDF documents not found, using fallback")
        # Fallback to original documents if available
        if 'documents' in globals():
            source_docs = documents
        else:
            source_docs = ["This is a test document for demonstration purposes."]
        
    print(f"📄 Working with {len(source_docs)} document(s)")
    
    # Show a sample of what we're working with
    if source_docs:
        sample = source_docs[0][:300] + "..." if len(source_docs[0]) > 300 else source_docs[0]
        print(f"📝 Sample content: {sample}")
    
except Exception as e:
    print(f"❌ Error accessing documents: {e}")
    source_docs = ["This is a test document for demonstration purposes."]

# 2. Split documents
print(f"\n🔄 Splitting documents...")
try:
    text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    split_docs = text_splitter.split_texts(source_docs)
    print(f"✅ Created {len(split_docs)} chunks")
    
    demo_chunks = split_docs[:15]
    print(f"📊 Using {len(demo_chunks)} chunks for demo")
    
except Exception as e:
    print(f"❌ Error splitting documents: {e}")
    demo_chunks = ["Sample chunk 1", "Sample chunk 2", "Sample chunk 3"]

# 3. Create enhanced vector database
print(f"\n🗃️ Creating enhanced vector database...")
try:
    enhanced_db = SimpleEnhancedVectorDatabase()
    enhanced_db = asyncio.run(enhanced_db.abuild_from_list(demo_chunks))
    
    print(f"✅ Enhanced database created with {len(enhanced_db.vectors)} vectors")
    print(f"📏 Available metrics: {list(enhanced_db.distance_metrics.keys())}")
    
except Exception as e:
    print(f"❌ Error creating enhanced database: {e}")
    enhanced_db = None

# 4. Test all distance metrics
if enhanced_db:
    print(f"\n📏 Testing All Distance Metrics...")
    
    # Create a query that might match your content better
    test_queries = [
        "What is this document about?",
        "Tell me about the main topic",
        "What information is provided?"
    ]
    
    # Test with first query
    test_query = test_queries[0]
    print(f"🔍 Query: '{test_query}'")
    
    results_by_metric = {}
    
    for metric in ['cosine', 'euclidean', 'manhattan']:
        try:
            results = enhanced_db.search_with_metric(test_query, k=3, distance_measure=metric)
            if results:
                top_score = results[0][1]
                results_by_metric[metric] = results
                print(f" {metric.upper()}: {len(results)} results, top score: {top_score:.3f}")
                
                # Show top result snippet
                top_text = results[0][0]
                snippet = top_text[:80] + "..." if len(top_text) > 80 else top_text
                print(f"   📝 Top result: {snippet}")
            else:
                print(f" {metric.upper()}: No results")
                
        except Exception as e:
            print(f" {metric.upper()}: {e}")
    
    # 5. Compare metrics
    if results_by_metric:
        print(f"\n📊 METRIC COMPARISON:")
        print("-" * 30)
        for metric, results in results_by_metric.items():
            scores = [score for _, score in results]
            print(f"{metric.upper():>10}: {scores}")
        
    
    # 6. Test with existing RAG pipeline if available
    print(f"\n🤖 Testing with RAG Pipeline...")
    try:
        if 'chat_openai' in globals() and 'rag_system_prompt' in globals():
            # Create a simple RAG test
            rag_pipeline = RetrievalAugmentedQAPipeline(
                llm=chat_openai,
                vector_db_retriever=enhanced_db,
                response_style="detailed",
                include_scores=True
            )
            
            # Test it
            result = rag_pipeline.run_pipeline(test_query, k=3)
            print(f"{result['response'][:200]}...")
            print(f" Used {result['context_count']} sources")
            
        else:
            print(" RAG pipeline components not available")
            
    except Exception as e:
        print(f"RAG pipeline test failed: {e}")

else:
    print("Enhanced database not available for testing")



🚀 Complete Enhanced RAG Test
✅ Using 1 PDF document(s)
📄 Working with 1 document(s)
📝 Sample content: 
--- Page 1 ---
Matryoshka Representation Learning
Aditya Kusupati∗†⋄, Gantavya Bhatt∗†, Aniket Rege∗†,
Matthew Wallingford†, Aditya Sinha⋄, Vivek Ramanujan†, William Howard-Snyder†,
Kaifeng Chen⋄, Sham Kakade‡, Prateek Jain⋄and Ali Farhadi†
†University of Washington,⋄Google Research,‡Harvard Univer...

🔄 Splitting documents...
✅ Created 182 chunks
📊 Using 15 chunks for demo

🗃️ Creating enhanced vector database...
✅ Enhanced database created with 15 vectors
📏 Available metrics: ['cosine', 'euclidean', 'manhattan']

📏 Testing All Distance Metrics...
🔍 Query: 'What is this document about?'
 COSINE: 3 results, top score: 0.307
   📝 Top result: l9x40IRt/6GO//GTDsLrR4IHM65l3ty/JgzpR3nyyrMzS8sLhWXSyura+sb9uZWU0WJJLRBIh7Jto8V5...
 EUCLIDEAN: 3 results, top score: 0.459
   📝 Top result: l9x40IRt/6GO//GTDsLrR4IHM65l3ty/JgzpR3nyyrMzS8sLhWXSyura+sb9uZWU0WJJLRBIh7Jto8V5...
 MANHATTAN: 3 results, 